# Aula Prática: Alinhando uma Amostra de E. coli a um Genoma de Referência

## Ferramentas que usaremos:

* Google Colab: Para executar a análise na nuvem.

* Biopython: Para manipular dados de sequência.

* Bowtie2: Para alinhamento.

* SAMtools: Para processar arquivos de alinhamento.

* Pysam: Para visualizar resultados de alinhamento.

In [ ]:
# Instalação das ferramentas necessárias
!apt update
!apt install -y bowtie2 samtools
!pip install biopython pysam
!apt install -y bcftools

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
32 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
# Faça o download do genoma de referencia que usaremos: E. coli K-12 genoma de referencia do NCBI (https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/).
# Download E. coli genoma de referencia (K-12 cepa)
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz

# Renomeando
!mv GCF_000005845.2_ASM584v2_genomic.fna ecoli_ref.fna

# Indexação do genoma de referencia com o Bowtie2
!bowtie2-build ecoli_ref.fna ecoli_ref

--2025-03-17 20:02:35--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  4.77MB/s    in 0.3s    

2025-03-17 20:02:38 (4.77 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]

Settings:
  Output files: "ecoli_ref.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024


In [ ]:
# E. coli disponível publicamente do Sequence Read Archive (SRA); SRA SRR12345678
# Instale o SRA toolkit para fazer o download da amostra
!apt install -y sra-toolkit

# Download da amostra
!prefetch SRR12345678

# A amostra será baixada como arquivos FASTQ (por exemplo, SRR12345678_1.fastq e SRR12345678_2.fastq para leituras de extremidade pareada)
!fasterq-dump SRR12345678

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
Suggested packages:
  blends-doc menu-l10n gksu | kde-runtime | ktsuss
The following NEW packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu sra-toolkit
0 upgraded, 7 newly installed, 0 to remove and 32 not upgraded.
Need to get 8,290 kB of archives.
After this operation, 23.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 menu amd64 2.1.47ubuntu4 [354 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 blends-common all 0.7.4ubuntu1 [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libkdf5-2 amd64 2.11.2+dfsg-4build2 [14.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libncbi-vdb2 amd64 2.11.2+dfsg-4build2 [1,364 kB]
Get:5

In [ ]:
# Run Bowtie2 alinhamento
!bowtie2 -x ecoli_ref -1 SRR12345678_1.fastq -2 SRR12345678_2.fastq -S aligned_output.sam

# Converção do formato SAM para BAM
!samtools view -bS aligned_output.sam -o aligned_output.bam

# 'Sort' do arquivo BAM
!samtools sort aligned_output.bam -o aligned_output_sorted.bam

# Indexação do arquivo BAM organizado 'sorted'
!samtools index aligned_output_sorted.bam

23655 reads; of these:
  23655 (100.00%) were paired; of these:
    23526 (99.45%) aligned concordantly 0 times
    0 (0.00%) aligned concordantly exactly 1 time
    129 (0.55%) aligned concordantly >1 times
    ----
    23526 pairs aligned concordantly 0 times; of these:
      0 (0.00%) aligned discordantly 1 time
    ----
    23526 pairs aligned 0 times concordantly or discordantly; of these:
      47052 mates make up the pairs; of these:
        46461 (98.74%) aligned 0 times
        0 (0.00%) aligned exactly 1 time
        591 (1.26%) aligned >1 times
1.79% overall alignment rate


In [ ]:
# Visualização do alinhamento
import pysam

# Open the sorted BAM file
bamfile = pysam.AlignmentFile("aligned_output_sorted.bam", "rb")

# Print the first few alignments
for read in bamfile.head(5):
    print(read)

SRR12345678.200	73	NC_000913.3	224304	0	151M	NC_000913.3	224304	0	TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTTGTAAGACAGATGTGAAATGCCCGGGCTTAACCTGGGAACTGCATTTGTGACTGCAAGGCTAGAATCTGGCAGAGGGGGGTAGAATTCCACGT	array('B', [34, 34, 34, 34, 34, 37, 37, 34, 34, 37, 33, 32, 38, 38, 38, 38, 38, 38, 38, 38, 37, 38, 39, 38, 38, 38, 36, 37, 39, 39, 39, 39, 38, 39, 39, 39, 38, 38, 38, 38, 39, 39, 39, 38, 38, 35, 36, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 16, 33, 38, 39, 39, 39, 39, 18, 37, 39, 39, 39, 38, 37, 38, 39, 39, 39, 39, 36, 39, 39, 36, 31, 36, 38, 38, 39, 39, 39, 39, 39, 39, 30, 38, 37, 38, 39, 38, 39, 39, 39, 37, 38, 39, 39, 38, 38, 17, 38, 39, 37, 38, 39, 37, 38, 17, 37, 36, 38, 37, 39, 39, 39, 37, 37, 39, 37, 39, 39, 39, 38, 38, 36, 37, 32, 34, 30, 37, 26, 32, 36, 37, 36, 37, 37, 37, 37, 33, 37, 35, 36])	[('AS', -89), ('XN', 0), ('XM', 19), ('XO', 0), ('XG', 0), ('NM', 19), ('MD', '4G39C0A12G0T4T12C8C16C2A0T4G0C1A3T2G3C1T19G2'), ('YT', 'UP')]
SRR12345678.200	133	NC_0

In [ ]:
# Calculando a estatística do alinhamento

!samtools flagstat aligned_output_sorted.bam

47310 + 0 in total (QC-passed reads + QC-failed reads)
47310 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
849 + 0 mapped (1.79% : N/A)
849 + 0 primary mapped (1.79% : N/A)
47310 + 0 paired in sequencing
23655 + 0 read1
23655 + 0 read2
258 + 0 properly paired (0.55% : N/A)
258 + 0 with itself and mate mapped
591 + 0 singletons (1.25% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [ ]:
# Gerando o arquivo pileup
!samtools mpileup -f ecoli_ref.fna aligned_output_sorted.bam > alignment.pileup

[mpileup] 1 samples in 1 input files


In [ ]:
# Chamada de variantes: gerando o arquivo VCF
!bcftools mpileup -f ecoli_ref.fna aligned_output_sorted.bam | bcftools call -mv -o variants.vcf

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


In [ ]:
!head -n 30 variants.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.13+htslib-1.13+ds
##bcftoolsCommand=mpileup -f ecoli_ref.fna aligned_output_sorted.bam
##reference=file://ecoli_ref.fna
##contig=<ID=NC_000913.3,length=4641652>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPBZ,Number=1,Type=Float,Description="Mann-Whitney U-z test of Read Position Bias (closer to 0 is better)">
##INFO=<ID=MQBZ,Number=1,Type=Float,Descriptio

In [ ]:
# Filtrando o arquivo VCF
!bcftools filter -i 'QUAL > 3' variants.vcf -o filtered_variants.vcf

In [ ]:
!head -n 38 filtered_variants.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.13+htslib-1.13+ds
##bcftoolsCommand=mpileup -f ecoli_ref.fna aligned_output_sorted.bam
##reference=file://ecoli_ref.fna
##contig=<ID=NC_000913.3,length=4641652>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of raw reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of raw reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPBZ,Number=1,Type=Float,Description="Mann-Whitney U-z test of Read Position Bias (closer to 0 is better)">
##INFO=<ID=MQBZ,Number=1,Type=Float,Descriptio

In [ ]:
import pysam

# Abra o arquivo VCF:
vcf = pysam.VariantFile("filtered_variants.vcf")

# Imprima as primeiras variantes
for record in vcf:
    print(record)
    #break  # Remova esta linha para imprimir mais variantes

NC_000913.3	2730476	.	C	G	5.75677	PASS	DP=104;VDB=1;SGB=-0.693147;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=0,0,23,29;MQ=0	GT:PL	1/1:34,157,0

NC_000913.3	2730496	.	A	G	3.78127	PASS	DP=104;VDB=1;SGB=-0.693147;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=0,0,24,28;MQ=0	GT:PL	1/1:31,154,0

NC_000913.3	2730498	.	G	C	5.75677	PASS	DP=104;VDB=1;SGB=-0.693147;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=0,0,23,29;MQ=0	GT:PL	1/1:34,157,0

NC_000913.3	2730500	.	G	T	3.78393	PASS	DP=104;VDB=1;SGB=-0.693147;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=0,0,23,29;MQ=0	GT:PL	1/1:31,151,0

NC_000913.3	2730505	.	A	T	5.75677	PASS	DP=104;VDB=1;SGB=-0.693147;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=0,0,23,29;MQ=0	GT:PL	1/1:34,157,0

NC_000913.3	2730510	.	T	C	5.70756	PASS	DP=104;VDB=1;SGB=-0.693147;RPBZ=-1.12288;BQBZ=2.17321;FS=0;MQ0F=0.5;AC=2;AN=2;DP4=1,0,22,29;MQ=0	GT:PL	1/1:31,136,0

NC_000913.3	2730517	.	A	C	12.5751	PASS	DP=104;VDB=0.999824;SGB=-0.693146;RPBZ=-1.28338;BQBZ=0.856753;FS=0;MQ0F=0.432692;AC=2;AN=2;DP4=1,0,16,28;MQ=0	GT:PL	1/1:39,119,0

NC_000913.3	3428122	.	A	C	9.70

# Etapa: Visualizar variantes

Você também pode visualizar as variantes no IGV (Integrative Genomics Viewer):

Baixe o arquivo variants.vcf do Colab para sua máquina local.

Abra o IGV e carregue o genoma de referência (ecoli_ref.fna) e o arquivo VCF.

# Referências adicionais
https://www.deepseek.com/